## imports

In [ ]:
import re

In [ ]:
!pip3 install python-rake

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.9 MB/s 
     |████████████████████████████████| 8.2 MB 3.1 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=5ce7f190dfda285742801387c84a4e0091d4a9912e3d7d0a6643d970e60c1c8a
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt


In [60]:
!pip install summa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54 kB 2.5 MB/s 
  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54410 sha256=80f11f52917ff615f33de81fee45ba811bb409099ef1d910125936d5a7294f72
  Stored in directory: /root/.cache/pip/wheels/29/64/ac/7b443477588d365ef37ada30d456bdf5f07dc5be9f6324cb6e
Successfully built summa


In [65]:
!pip install yake

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 60 kB 4.2 MB/s 
     |████████████████████████████████| 132 kB 22.9 MB/s 
  Created wheel for jellyfish: filename=jellyfish-0.9.0-cp37-cp37m-linux_x86_64.whl size=73986 sha256=dbe043f222e318010f6e411aa2b6ab0dec8886f0940888a0b92d5e1ec0c4d1cc
  Stored in directory: /root/.cache/pip/wheels/fe/99/4e/646ce766df0d070b0ef04db27aa11543e2767fda3075aec31b
Successfully built jellyfish


In [ ]:
import RAKE
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
stop = stopwords.words('english')

In [ ]:
from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize

In [ ]:
m = MorphAnalyzer()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from numpy import array, log
vectorizer = CountVectorizer()

In [ ]:
from tqdm import tqdm

In [61]:
import summa

In [66]:
import yake

## dataset

В качестве корпуса беру рандомные тексты из [ленты на хабре](https://habr.com/en/all/), т.к. там уже размечены ключевые слова. Поскольку ключевые слова, как правило, являются терминами и приводятся по-английски, то и статьи буду брать только на английском

In [ ]:
with open('texts.txt', 'r', encoding='utf-8') as file:
  text = file.read()
  texts = text.split('=====')
len(texts)

7

In [ ]:
texts.pop(-1)

''

ключевые слова берем из тэгов, они выглядят как перечисленные через запятую термины. чтобы сделать из этого списки, рассплитим по запятой

In [ ]:
with open('keywords.txt', 'r', encoding='utf-8') as file:
  k_text = file.read()
  keywords = k_text.split('=====')
len(keywords)

7

In [ ]:
keywords.pop(-1)

''

In [ ]:
keywords = [x.split(',') for x in keywords]

In [ ]:
keywords

[['mobil',
  'emobile first',
  'mobile development',
  'mobile design',
  'mobile apps',
  'mobile web',
  'start-up',
  'startups',
  'business',
  'flow\n'],
 ['\nui/ux', 'interfaces', 'data-driven design\n'],
 ['\nkafka',
  'rabbitmq',
  'message',
  'message broker',
  'message queue',
  'performance',
  'fault tolerance\n'],
 ['\nguitar', 'distorton', 'amplifier', 'guitar effects', 'circuit design\n'],
 ['\nai',
  'stable diffusion',
  'stablediffusion',
  'invokeai',
  'neural networks',
  'neural paintings',
  'image processing',
  'graphic design',
  'art',
  'artificial intelligence\n'],
 ['\napple', 'ar', 'vr\n']]

убираем ненужные переносы строки

In [ ]:
for i in keywords:
  for ii in range(len(i)):
    i[ii] = re.sub('\n', '', i[ii])

In [ ]:
keywords

[['mobil',
  'emobile first',
  'mobile development',
  'mobile design',
  'mobile apps',
  'mobile web',
  'start-up',
  'startups',
  'business',
  'flow'],
 ['ui/ux', 'interfaces', 'data-driven design'],
 ['kafka',
  'rabbitmq',
  'message',
  'message broker',
  'message queue',
  'performance',
  'fault tolerance'],
 ['guitar', 'distorton', 'amplifier', 'guitar effects', 'circuit design'],
 ['ai',
  'stable diffusion',
  'stablediffusion',
  'invokeai',
  'neural networks',
  'neural paintings',
  'image processing',
  'graphic design',
  'art',
  'artificial intelligence'],
 ['apple', 'ar', 'vr']]

проверяем что в корпусе достаточное количество токенов:

In [ ]:
words = re.findall('\w+', text)
len(words)

8494

golden corpus

вручную размечаю тексты по ключевым словам и записываю в отдельный файл

In [ ]:
with open('golden_keywords.txt', 'r', encoding='utf-8') as file:
  gk_text = file.read()
  g_keywords = gk_text.split('=====')
len(g_keywords)

6

In [ ]:
g_keywords

['start-up,app,mobile apps,business,customers\n',
 '\nsystem,design,production,machines\n',
 '\nkafka,rabbitmq,message,apps,broker\n',
 '\nmusic,modeling,distortion,guitar\n',
 '\nneural network,image\n',
 '\napple,ar/vr,rumor']

In [ ]:
g_keywords = [x.split(',') for x in g_keywords]

In [ ]:
for i in g_keywords:
  for ii in range(len(i)):
    i[ii] = re.sub('\n', '', i[ii])

In [ ]:
g_keywords

[['start-up', 'app', 'mobile apps', 'business', 'customers'],
 ['system', 'design', 'production', 'machines'],
 ['kafka', 'rabbitmq', 'message', 'apps', 'broker'],
 ['music', 'modeling', 'distortion', 'guitar'],
 ['neural network', 'image'],
 ['apple', 'ar/vr', 'rumor']]

моя разметка во многом совпадает с разметкой хабра, но я решила, что будет еще лучше, если мы их объединим

In [ ]:
all_keywords = []
for i in range(len(keywords)):
  all_keywords.append(set(keywords[i])|set(g_keywords[i]))

In [ ]:
all_keywords

[{'app',
  'business',
  'customers',
  'emobile first',
  'flow',
  'mobil',
  'mobile apps',
  'mobile design',
  'mobile development',
  'mobile web',
  'start-up',
  'startups'},
 {'data-driven design',
  'design',
  'interfaces',
  'machines',
  'production',
  'system',
  'ui/ux'},
 {'apps',
  'broker',
  'fault tolerance',
  'kafka',
  'message',
  'message broker',
  'message queue',
  'performance',
  'rabbitmq'},
 {'amplifier',
  'circuit design',
  'distortion',
  'distorton',
  'guitar',
  'guitar effects',
  'modeling',
  'music'},
 {'ai',
  'art',
  'artificial intelligence',
  'graphic design',
  'image',
  'image processing',
  'invokeai',
  'neural network',
  'neural networks',
  'neural paintings',
  'stable diffusion',
  'stablediffusion'},
 {'apple', 'ar', 'ar/vr', 'rumor', 'vr'}]

## извлечение ключевых слов

пробуем извлечение ключевых слов разными способами

In [ ]:
rake = RAKE.Rake(stop)

In [ ]:
def normalize_text(text):
    lemmas = []
    for t in simple_word_tokenize(text):
        lemmas.append(
            m.parse(t)[0].normal_form
        )
    return ' '.join(lemmas)

In [ ]:
out_rake = []
for t in texts:
  out_rake.append(rake.run(normalize_text(t), maxWords=3, minFrequency=2))

In [ ]:
out_rake

[[('pivot sketch made', 6.833333333333334),
  ('still available', 4.25),
  ('mobile apps', 4.201680672268907),
  ('crossfit event', 4.0),
  ('bank acquiring', 4.0),
  ('gantt chart', 4.0),
  ('phone number', 4.0),
  ('mobile first', 3.9754901960784315),
  ('beauty uber', 3.9523809523809526),
  ('technical challenges', 3.8333333333333335),
  ('target audience', 3.75),
  ('mobile app', 3.6977124183006533),
  ('product manager', 3.666666666666667),
  ('beauty masters', 3.6357142857142857),
  ('little later', 3.583333333333333),
  ('google calendar', 3.425),
  ('main one', 3.378787878787879),
  ('customers ver', 3.357142857142857),
  ('masters ver', 3.35),
  ('webview app', 3.3055555555555554),
  ('customer app', 3.1626984126984126),
  ('master app', 2.914750957854406),
  ('one startup', 2.878787878787879),
  ('go', 2.0),
  ('founder', 2.0),
  ('business', 1.8571428571428572),
  ('android', 1.75),
  ('idea', 1.75),
  ('like', 1.75),
  ('article', 1.6666666666666667),
  ('created', 1.666666

text rank

In [62]:
trank_kw = [set([i[0] for i in summa.keywords.keywords(text, ratio=0.1, language='english', additional_stopwords=stop, scores=True)]) for text in texts]

In [64]:
trank_kw

[{'android',
  'api',
  'app',
  'apps',
  'bar',
  'beauty',
  'book',
  'booking',
  'business',
  'businesses',
  'calendar',
  'certain number',
  'challenge',
  'challenges',
  'consumables',
  'customer',
  'customers',
  'develop',
  'developer',
  'developers',
  'developing',
  'development',
  'directly',
  'high',
  'highly',
  'home',
  'later',
  'like',
  'liked',
  'likely',
  'main',
  'manage',
  'managed',
  'management',
  'manager',
  'managers',
  'master',
  'masters',
  'new',
  'option',
  'options',
  'order',
  'ordering',
  'orders',
  'personal',
  'personally',
  'pivot',
  'pivots',
  'price',
  'prices',
  'right',
  'service',
  'services',
  'time',
  'times',
  'timing',
  'user',
  'users',
  'ver',
  'web',
  'work',
  'worked',
  'working',
  'works'},
 {'control problems',
  'controlling',
  'data',
  'design',
  'designed',
  'designs',
  'downtime',
  'enterprise',
  'equipment',
  'industrial',
  'interfaces',
  'plan',
  'planning',
  'problem'

yake

In [67]:
yk = yake.KeywordExtractor(lan='en', n=1)
yake_kw = [set([i[0] for i in yk.extract_keywords(text)]) for text in texts]

In [68]:
yake_kw

[{'Android',
  'Beauty',
  'Calendar',
  'Google',
  'app',
  'apps',
  'booking',
  'business',
  'customer',
  'customers',
  'made',
  'master',
  'masters',
  'mobile',
  'order',
  'service',
  'services',
  'startup',
  'time',
  'work'},
 {'Data',
  'Internet',
  'Team',
  'Things',
  'design',
  'efficiency',
  'enterprise',
  'equipment',
  'industrial',
  'interface',
  'machines',
  'monitoring',
  'plant',
  'process',
  'processes',
  'production',
  'section',
  'system',
  'user',
  'users'},
 {'Consumer',
  'Data',
  'Infrastructure',
  'Kafka',
  'Patterns',
  'Programming',
  'Queue',
  'RabbitMQ',
  'Topic',
  'app',
  'broker',
  'brokers',
  'communication',
  'consumers',
  'delivery',
  'message',
  'messages',
  'read',
  'recipient',
  'sender'},
 {'Circuit',
  'DSP',
  'Digital',
  'Software',
  'Sound',
  'TINA',
  'amplifiers',
  'analog',
  'circuits',
  'devices',
  'distortion',
  'electric',
  'guitar',
  'idea',
  'op-amp',
  'output',
  'signal',
  'tu

## шаблоны